# Journal2Bord

- 17.04: Retook on JR's pseudocode, in order to make it work with the dataset, and fix some misalignment issues (matchlist needed when re-epoching, index_resetting needed, and also the use of mne_events saves a lot of time)

*Next: run it on Neurospin's workstation on a few more subjects, as it takes too much time on laptop.*

- 18.04: Fixed more queries handling and general structure, and it runs on Neurospin Workstation.
Pb: For one subject, one run, and decoding one feature (word embeddings), it takes around 1h...
Added also the averaged on subject 

- 19.04: Fixed all the problems discussed during our weekly meeting (decim, meta handling, preload, etc..).

Came up with a way to generate the plots for all conditions, added a better epochs window handling.

- 20.04: Trying to reduce the RAM usage, by calculating the evos & scores directly, not keeping the epochs in memory.
Created script for JeanZay, running jobs successfully now. 
Plot generated for different sliding windows, multiple subjects, all runs. 

- 21.04: Tried to adapt the newly parsed files for the syntax, adapted but the problem is that the parser has not been run on the correct LPP translation.. TODO contact the person in charge and rerun the parser

Also, adapted script for Jean Zay: sent email for support to understand the memory faults.

- 26.04: After iterating on Jean Zay: there is a problem with the way the offset of the different modalities is handled. The way it is currently done, there is a shift: the offset is the offset of the previous word/sent/const, instead of being the end of the one we care about. 

Next steps include:
Ongoing / Done
- Pausing the testing on Jean Zay and instead focusing on testing locally with first: only word on/offset and decim * 100.
- 
- Changing the mne_events function so that we can give in the parameter sent/const/word and 
- Adding the info about the sent_id/sent_stop in the metadata using cumsum / groupby

Next:

- check it worked for words with the decoding

## TODO important

- Check if the sentence_stop makes sense: currently done by taking the max of the subgroup (groupby sentence_id) and add the duration of the last word

- Ask JR about look of word onset/offset decoding

TODO another time:
- investigate the events_repeated
- train on subset for words, and decode on other modalities ?
- If the script runs correctly on Jean Zay, add other decoding modalities, and run them.


# Testing new syntactic parser

### Removing regex punct


In [14]:
import re
for i in range(1,10):
    
    # Read the contents of the text file
    with open(f"/home/co/code/data/syntax_new_untested/run{i}_v2_0.25_0.5-tokenized.syntax.txt", "r") as file:
        text = file.read()

    # Define the pattern to match substrings like (PONCT 3=,)
    pattern = r'\(PONCT\s\d+.*\)'
    # Remove the substrings using re.sub()
    clean_text = re.sub(pattern, '', text)

    clean_text
    # Write the cleaned text back to the file
    with open(f"/home/co/code/data/syntax_new_no_punct/run{i}_v2_0.25_0.5-tokenized.syntax.txt", "w") as file:
        file.write(clean_text)


In [15]:
clean_text

"(SENT (VN (CLS-SUJ 0=Il) (CLO 1=y) (V 2=avait)) \n(SENT (Ssub-MOD (CS 0=Lorsque) (Sint (VN (CLS-SUJ 1=je) (V 2=revins)) (PP-DE_OBJ (P 3=de) (NP (DET 4=mon) (NC 5=travail))))) \n(SENT (COORD (CC 0=Et) (Sint (VN (CLS-SUJ 1=je) (CLO-OBJ 2=l') (V 3=entendis)) (Srel-MOD (NP-SUJ (PROREL 4=qui)) (VN (V 5=parlait))))) \n(SENT (VN (CLS-SUJ 0=tu) (ADV 1=ne) (CLR 2=t') (CLO 3=en) (V 4=souviens)) (ADV 5=donc) (ADV 6=pas) \n(SENT (VN (V 0=Disait) (CLS-SUJ 1=il)) \n(SENT (VN (CLS-SUJ 0=Ce) (ADV 1=n') (V 2=est)) (ADV+ (ADV 3=pas) (ADV 4=tout) (P 5=à) (NC 6=fait)) (ADV 7=ici!))\n(SENT (NP-SUJ (DET 0=Une) (ADJ 1=autre) (NC 2=voix)) (VN (CLO-A_OBJ 3=lui) (V 4=répondit)) (ADV+ (P 5=sans) (NC 6=doute)) \n(Ssub (CS 0=si!))\n(Ssub (CS 0=Si!))\n(SENT (VN (CLS-SUJ 0=C') (V 1=est)) (ADV 2=bien) (NP-ATS (DET 3=le) (NC 4=jour)) \n(SENT (VN (VN (CLS-SUJ 0=Je) (ADV 1=ne) (V 2=voyais)) (COORD (CC 3=ni) (VN (ADV 4=n') (V 5=entendais)))) (ADV 6=toujours) (NP-OBJ (NC 7=personne)) \n(SENT (ADV 0=Pourtant) (NP-SUJ (DET

In [16]:
def mne_events(meta, raw, start, level):
    if start=='onset':
        events = np.ones((len(meta), 3), dtype=int)
        events[:, 0] = meta.start * raw.info["sfreq"]
        return dict(events=events, metadata=meta.reset_index())
    elif start=='offset':
        events = np.ones((len(meta), 3), dtype=int)
        events[:, 0] = (meta.start+meta.duration) * raw.info["sfreq"]
        return dict(events=events, metadata=meta.reset_index())
        
    else:
        print('start should be either onset or offset')
        return 0
from dataset import read_raw, get_subjects, get_path
from utils import decod_xy
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy
nlp = spacy.load("fr_core_news_sm")

all_evos = []
all_scores = []

path = get_path("LPP_read")
subjects = get_subjects(path)
task = "read"
# Debug
runs = 1

epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}

levels = ('word')
starts = ('onset')
            
for subject in subjects[2:3]:
    dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
    # Dict init
    for start in starts: 
            for level in levels:
                epoch_key = f'{level}_{start}'
                dict_epochs[epoch_key] = [] 
    for run in range(1,runs+1):
        raw, meta_, events = read_raw(subject, run, events_return = True)
        meta = meta_.copy()
        # Metadata update
        # Word start
        meta['word_onset'] = True
        meta['word_stop'] = meta.start + meta.duration

        # Sent start
        meta['sentence_onset'] = meta.word_id == 0

        # Const start
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)
        
        # Adding the sentence stop info
        meta['sentence_id'] = np.cumsum(meta.sentence_onset)
        for s, d in meta.groupby('sentence_id'):
            meta.loc[d.index, 'sent_word_id'] = range(len(d))
            meta.loc[d.index, 'sentence_start'] = d.start.min()
            meta.loc[d.index, 'sentence_stop'] = d.start.max() # TO Verify!
            
        # Adding the constituents stop info
        meta['constituent_id'] = np.cumsum(meta.constituent_onset)
        for s, d in meta.groupby('constituent_id'):
            meta.loc[d.index, 'constituent_start'] = d.start.min()
            meta.loc[d.index, 'constituent_stop'] = d.start.max() # TO Verify!
            meta.loc[d.index, 'const_word_id'] = range(len(d))



 Epoching for run 1, subject: 5

Opening raw data file /home/co/data/LPP_MEG_visual/sub-5/ses-01/meg/sub-5_ses-01_task-read_run-01_meg.fif...
    Read a total of 13 projection items:
        grad_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v6 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v7 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v8 (1 x 306)  idle
    Range : 89000 ... 554999 =     89.000 ...   554.999 secs
Ready.
Reading events from /home/co/data/LPP_MEG_visual/sub-5

/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:51: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:51: RuntimeWarning: Omitted 128 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:51: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne_bids.read_raw_bids(bids_path)
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:51: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012

1468 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Reading 0 ... 465999  =      0.000 ...   465.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 6601 samples (6.601 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 306 out of 306 | elapsed:   25.0s finished


In [17]:
meta[['word', 'constituent_id','n_closing','const_word_id']][:50]

,word,constituent_id,n_closing,const_word_id
0,Lorsque,0,1,0.0
1,javais,0,1,1.0
2,six,0,2,2.0
3,ans,1,1,0.0
4,jai,1,1,1.0
5,vu,1,1,2.0
6,une,1,1,3.0
7,fois,1,1,4.0
8,une,1,2,5.0
9,magnifique,2,1,0.0


# Initial Plotting for ERPs

### 0.5

Just the evoked, for all conditions, all subjects:



In [ ]:
# It is currently being done by keeping all the epochs in memory: might want to do like in the #1, and generate the evo
# or score from the epochs (for a subject), and from there try
# To find a way to, starting with an array of evoked, average them!!!
from dataset import read_raw, get_subjects, get_path, mne_events
from utils import decod_xy
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy
nlp = spacy.load("fr_core_news_sm")

all_evos = []

path = get_path("LPP_read")
subjects = get_subjects(path)
task = "read"
# Debug
runs = 9

epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}

dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
# Dict init
for start in ('onset', 'offset'): 
        for level in ('word', 'constituent', 'sentence'):
            epoch_key = f'{level}_{start}'
            dict_epochs[epoch_key] = [] 
            
for subject in subjects[2:5]:
    all_epochs = []
    for run in range(1,runs+1):
        raw, meta_, events = read_raw(subject, run, events_return = True)
        meta = meta_.copy()
        # Metadata update
        # Word start
        meta['word_onset'] = True

        # Word end
        meta['word_offset'] = True

        # Sent start
        meta['sentence_onset'] = meta.word_id == 0

        # Sent stop
        meta['next_word_id'] = meta['word_id'].shift(-1)
        meta['sentence_offset'] = meta.apply(lambda x: True if x['word_id'] > x['next_word_id'] else False, axis=1)
        meta['sentence_offset'].fillna(False, inplace=True)
        meta.drop('next_word_id', axis=1, inplace=True)

        # Const start
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: True if x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1 else False, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)

        # Const stop
        meta['next_closing'] = meta['n_closing'].shift(-1)
        meta['constituent_offset'] = meta.apply(lambda x: True if x['n_closing'] > x['next_closing'] else False, axis=1)
        meta['constituent_offset'].fillna(False, inplace=True)
        meta.drop('next_closing', axis=1, inplace=True)

        for start in ('onset', 'offset'): 
            # for level in ('word', 'constituent', 'sentence'):
            for level in ('sentence', 'constituent', 'word'):
                # Select only the rows containing the True for the conditions (sentence_end, etc..)
                sel = meta.query(f'{level}_{start}==True')
                assert sel.shape[0] > 10  #
                # TODO check variance as well for sentences
                # Matchlist events and meta
                # So that we can epoch now that's we've sliced our metadata
                i, j = match_list(events[:, 2], sel.word.apply(len))
                sel = sel.reset_index().loc[j]
                epochs = mne.Epochs(raw, **mne_events(sel, raw), decim = 10,
                                     tmin = epoch_windows[f'{level}'][f'{start}_min'],
                                       tmax = epoch_windows[f'{level}'][f'{start}_max'],
                                         event_repeated = 'drop',
                                            preload=True)  # n_words OR n_constitutent OR n_sentences
                epoch_key = f'{level}_{start}'
            
                dict_epochs[epoch_key].append(epochs)

# Once we have the dict of epochs per condition full, we can concatenate them, and fix the dev_head             
for start_ in ('onset', 'offset'): 
    for level_ in ('word', 'constituent', 'sentence'):
        epoch_key = f'{level_}_{start_}'
        all_epochs_chosen = dict_epochs[epoch_key]
        # Concatenate epochs
        for epo in all_epochs_chosen:
            epo.info["dev_head_t"] = all_epochs_chosen[1].info["dev_head_t"]

        dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)
            
dict_evos = dict() # DICT containing epochs grouped by conditions (start x level)
# Dict init
for start in ('onset', 'offset'): 
        for level in ('word', 'constituent', 'sentence'):
            epoch_key = f'{level}_{start}'
            dict_evos[epoch_key] = [] 

# Now that we have all the epochs, rerun the plotting / decoding on averaged
for start in ('onset', 'offset'): 
        for level in ('word', 'constituent', 'sentence'):  
            epoch_key = f'{level}_{start}'
            epochs = dict_epochs[epoch_key]
            # mean
            evo = epochs.copy().pick_types(meg=True).average(method='median')
            dict_evos[epoch_key] = evo


### Plotting

In [ ]:
for level in ('word', 'constituent', 'sentence'):
    for start in ('onset', 'offset'):        
            epoch_key = f'{level}_{start}'
            print(f"Plotting for: {epoch_key}")
            dict_evos[epoch_key].plot(gfp=True)


### #1
First plot: 3x2 plot, that shows:
- From the onset, and offset of {word, constituent, sentence}:

The evoked potential linked to it, as well as the decoding of the word embedding

In [ ]:
def mne_events(meta, raw, start, level):
    if start=='onset':
        events = np.ones((len(meta), 3), dtype=int)
        events[:, 0] = meta.start * raw.info["sfreq"]
        return dict(events=events, metadata=meta.reset_index())
    elif start=='offset':
        """
        It should generalize, no need for different cases?
        if level == 'word':
            events = np.ones((len(meta), 3), dtype=int)
            events[:, 0] = (meta.start+meta.duration) * raw.info["sfreq"]
            return dict(events=events, metadata=meta.reset_index())
        elif level == 'sentence':
            events = np.ones((len(meta), 3), dtype=int)
            events[:, 0] = (meta.start+meta.duration) * raw.info["sfreq"]
            return dict(events=events, metadata=meta.reset_index())

        else:
            print('hi')
            # Fill
        """
        events = np.ones((len(meta), 3), dtype=int)
        events[:, 0] = (meta.start+meta.duration) * raw.info["sfreq"]
        return dict(events=events, metadata=meta.reset_index())
        
    else:
        print('start should be either onset or offset')
        return 0
        
        


# Test on words offset only # Done

# Generalize

In [ ]:
from dataset import read_raw, get_subjects, get_path
from utils import decod_xy
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy
nlp = spacy.load("fr_core_news_sm")

all_evos = []
all_scores = []

path = get_path("LPP_read")
subjects = get_subjects(path)
task = "read"
# Debug
runs = 9

epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}

levels = ('word','constituent','sentence')
starts = ('onset', 'offset')
            
for subject in subjects[2:6]:
    dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
    # Dict init
    for start in starts: 
            for level in levels:
                epoch_key = f'{level}_{start}'
                dict_epochs[epoch_key] = [] 
    for run in range(1,runs+1):
        raw, meta_, events = read_raw(subject, run, events_return = True)
        meta = meta_.copy()
        # Metadata update
        # Word start
        meta['word_onset'] = True
        meta['word_stop'] = meta.start + meta.duration

        # Sent start
        meta['sentence_onset'] = meta.word_id == 0

        # Const start
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)
        
        # Adding the sentence stop info
        meta['sentence_id'] = np.cumsum(meta.sentence_onset)
        for s, d in meta.groupby('sentence_id'):
            meta.loc[d.index, 'sent_word_id'] = range(len(d))
            meta.loc[d.index, 'sentence_start'] = d.start.min()
            meta.loc[d.index, 'sentence_stop'] = d.start.max() # TO Verify!
            
        # Adding the constituents stop info
        meta['constituent_id'] = np.cumsum(meta.constituent_onset)
        for s, d in meta.groupby('constituent_id'):
            meta.loc[d.index, 'constituent_start'] = d.start.min()
            meta.loc[d.index, 'constituent_stop'] = d.start.max() # TO Verify!
            meta.loc[d.index, 'const_word_id'] = range(len(d))

        for start in starts: 
            # for level in ('word', 'constituent', 'sentence'):
            # for level in ('sentence', 'constituent', 'word'):
            for level in levels:
                
                # Select only the rows containing the True for the conditions
                # Simplified to only get for the onset: sentence onset epochs, constituent onset epochs,etc
                sel = meta.query(f'{level}_onset==True')
                assert sel.shape[0] > 10  #
                # TODO check variance as well for sentences
                # Matchlist events and meta
                # So that we can epoch now that's we've sliced our metadata
                i, j = match_list(events[:, 2], sel.word.apply(len))
                sel = sel.reset_index().loc[j]
                # Making sure there is not hidden bug when matching
                assert sel.shape[0] > 0.8 *  (meta.query(f'{level}_onset==True')).shape[0]

                # Epoching from the metadata having all onset events: if the start=Offset, the mne events
                # Function will epoch on the offset of each level instead of the onset
                # TODO: add adaptative baseline
                epochs = mne.Epochs(raw, **mne_events(sel, raw ,start=start, level=level), decim = 100,
                                     tmin = epoch_windows[f'{level}'][f'{start}_min'],
                                       tmax = epoch_windows[f'{level}'][f'{start}_max'],
                                         event_repeated = 'drop', # check event repeated
                                            preload=True,
                                                baseline=None)  # n_words OR n_constitutent OR n_sentences
                epoch_key = f'{level}_{start}'

                dict_epochs[epoch_key].append(epochs)
            
    # Once we have the dict of epochs per condition full (epoching for each run for a subject)
    # we can concatenate them, and fix the dev_head             
    for start_ in starts: 
        for level_ in levels:
            epoch_key = f'{level_}_{start_}'
            all_epochs_chosen = dict_epochs[epoch_key]
            # Concatenate epochs

            for epo in all_epochs_chosen:
                epo.info["dev_head_t"] = all_epochs_chosen[1].info["dev_head_t"]

            dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)

    # Now that we have all the epochs, rerun the plotting / decoding on averaged
    for start in starts: 
        for level in levels:
            epoch_key = f'{level}_{start}'
            epochs = dict_epochs[epoch_key]
            # mean
            evo = epochs.copy().pick_types(meg=True).average(method='median')
            all_evos.append(dict(subject=subject, evo=evo, start=start, level=level))


            # decoding word emb
            epochs = epochs.load_data().pick_types(meg=True, stim=False, misc=False)
            X = epochs.get_data()
            embeddings = epochs.metadata.word.apply(lambda word: nlp(word).vector).values
            embeddings = np.array([emb for emb in embeddings])
            R_vec = decod_xy(X, embeddings)
            scores = np.mean(R_vec, axis=1)

            for t, score in enumerate(scores):
                all_scores.append(dict(subject=subject, score=score, start=start, level=level, t=epochs.times[t]))

all_scores = pd.DataFrame(all_scores)
all_evos = pd.DataFrame(all_evos)

all_scores.to_csv('./score.csv')
all_evos.to_csv('./evos.csv')


In [ ]:
all_scores = pd.DataFrame(all_scores)
all_evos = pd.DataFrame(all_evos)

from matplotlib.pyplot import figure

figure(figsize=(16, 10), dpi=80)

fig, axes = plt.subplots(3, 2)

for axes_, level in zip( axes, levels):  
    for ax, start in zip( axes_, starts):  
        cond1 = all_scores.level==f'{level}'
        cond2 = all_scores.start==f'{start}'
        data = all_scores[ cond1 & cond2]
        y = []
        x = []
        for s, t in data.groupby('t'):
            score_avg = t.score.mean()
            y.append(score_avg)
            x.append(s)

        ax.plot(x,y)
        ax.set_title(f'{level} {start}')


In [ ]:
sel[['word','start']]

In [ ]:
from dataset import read_raw, get_subjects, get_path
from utils import decod_xy
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy
nlp = spacy.load("fr_core_news_sm")

all_evos = []
all_scores = []

path = get_path("LPP_read")
subjects = get_subjects(path)
task = "read"
# Debug
runs = 9

epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}


            
for subject in subjects[2]:
    dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
    # Dict init
    for start in ('onset', 'offset'): 
            for level in ('word', 'constituent', 'sentence'):
                epoch_key = f'{level}_{start}'
                dict_epochs[epoch_key] = [] 
    for run in range(1,2):
        raw, meta_, events = read_raw(subject, run, events_return = True)
        meta = meta_.copy()
        # Metadata update
        # Word start
        meta['word_onset'] = True
        meta['word_stop'] = meta.start + meta.duration

        # Sent start
        meta['sentence_onset'] = meta.word_id == 0

        # Const start
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)
        
        # Adding the sentence stop info
        meta['sentence_id'] = np.cumsum(meta.sentence_onset)
        for s, d in meta.groupby('sentence_id'):
            meta.loc[d.index, 'sent_word_id'] = range(len(d))
            meta.loc[d.index, 'sentence_start'] = d.start.min()
            meta.loc[d.index, 'sentence_stop'] = d.start.max()
            
        # Adding the constituents stop info
        meta['constituent_id'] = np.cumsum(meta.constituent_onset)
        for s, d in meta.groupby('constituent_id'):
            meta.loc[d.index, 'constituent_start'] = d.start.min()
            meta.loc[d.index, 'constituent_stop'] = d.start.max()
            meta.loc[d.index, 'const_word_id'] = range(len(d))
        print(meta.head(50))
        
        for start in ('onset', 'offset'): 
            # for level in ('word', 'constituent', 'sentence'):
            # for level in ('sentence', 'constituent', 'word'):
            level = 'word'
                
            # Select only the rows containing the True for the conditions
            # Simplified to only get for the onset: sentence onset epochs, constituent onset epochs,etc
            sel = meta.query(f'{level}_onset==True')
            assert sel.shape[0] > 10  #
            # TODO check variance as well for sentences
            # Matchlist events and meta
            # So that we can epoch now that's we've sliced our metadata
            i, j = match_list(events[:, 2], sel.word.apply(len))
            sel = sel.reset_index().loc[j]
            # Making sure there is not hidden bug when matching
            assert sel.shape[0] > 0.8 *  (meta.query(f'{level}_onset==True')).shape[0]

            # Epoching from the metadata having all onset events: if the start=Offset, the mne events
            # Function will epoch on the offset of each level instead of the onset
            # TODO: add adaptative baseline
            epochs = mne.Epochs(raw, **mne_events(sel, raw ,start=start, level=level), decim = 100,
                                 tmin = epoch_windows[f'{level}'][f'{start}_min'],
                                   tmax = epoch_windows[f'{level}'][f'{start}_max'],
                                     event_repeated = 'drop', # check event repeated
                                        preload=True,
                                            baseline=None)  # n_words OR n_constitutent OR n_sentences
            epoch_key = f'{level}_{start}'

            dict_epochs[epoch_key].append(epochs)
        

In [ ]:
epochs

In [ ]:

epochs.copy().pick_types(meg=True).average(method='median').plot()

In [ ]:
from dataset import read_raw, get_subjects, get_path, mne_events
from utils import decod_xy
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy
nlp = spacy.load("fr_core_news_sm")

all_evos = []
all_scores = []

path = get_path("LPP_read")
subjects = get_subjects(path)
task = "read"
# Debug
runs = 9

epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}


            
for subject in subjects[2:10]:
    dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
    # Dict init
    for start in ('onset', 'offset'): 
            for level in ('word', 'constituent', 'sentence'):
                epoch_key = f'{level}_{start}'
                dict_epochs[epoch_key] = [] 
    for run in range(1,runs+1):
        raw, meta_, events = read_raw(subject, run, events_return = True)
        meta = meta_.copy()
        # Metadata update
        # Word start
        meta['word_onset'] = True

        # Sent start
        meta['sentence_onset'] = meta.word_id == 0

        # Const start
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: True if x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1 else False, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)
        
        # Adding the sequence
        meta['sequence_id'] = np.cumsum(meta.is_last_word.shift(1, fill_value=False))
        for s, d in meta.groupby('sequence_id'):
            meta.loc[d.index, 'word_id'] = range(len(d))
        

        for start in ('onset', 'offset'): 
            # for level in ('word', 'constituent', 'sentence'):
            for level in ('sentence', 'constituent', 'word'):
                # Select only the rows containing the True for the conditions (sentence_end, etc..)
                sel = meta.query(f'{level}_{start}==True')
                assert sel.shape[0] > 10  #
                # TODO check variance as well for sentences
                # Matchlist events and meta
                # So that we can epoch now that's we've sliced our metadata
                i, j = match_list(events[:, 2], sel.word.apply(len))
                sel = sel.reset_index().loc[j]
                epochs = mne.Epochs(raw, **mne_events(sel, raw), decim = 100,
                                     tmin = epoch_windows[f'{level}'][f'{start}_min'],
                                       tmax = epoch_windows[f'{level}'][f'{start}_max'],
                                         event_repeated = 'drop', # check event repeated
                                            preload=True)  # n_words OR n_constitutent OR n_sentences
                epoch_key = f'{level}_{start}'
            
                dict_epochs[epoch_key].append(epochs)
        
            
    # Once we have the dict of epochs per condition full (epoching for each run for a subject)
    # we can concatenate them, and fix the dev_head             
    for start_ in ('onset', 'offset'): 
        for level_ in ('word', 'constituent', 'sentence'):
            epoch_key = f'{level_}_{start_}'
            all_epochs_chosen = dict_epochs[epoch_key]
            # Concatenate epochs
            for epo in all_epochs_chosen:
                epo.info["dev_head_t"] = all_epochs_chosen[1].info["dev_head_t"]

            dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)

    # Now that we have all the epochs, rerun the plotting / decoding on averaged
    for start in ('onset', 'offset'): 
            for level in ('word', 'constituent', 'sentence'):  
                epoch_key = f'{level}_{start}'
                epochs = dict_epochs[epoch_key]
                # mean
                evo = epochs.copy().pick_types(meg=True).average(method='median')
                all_evos.append(dict(subject=subject, evo=evo, start=start, level=level))


                # decoding word emb
                epochs = epochs.load_data().pick_types(meg=True, stim=False, misc=False)
                X = epochs.get_data()
                embeddings = epochs.metadata.word.apply(lambda word: nlp(word).vector).values
                embeddings = np.array([emb for emb in embeddings])
                R_vec = decod_xy(X, embeddings)
                scores = np.mean(R_vec, axis=1)

                for t, score in enumerate(scores):
                    all_scores.append(dict(subject=subject, score=score, start=start, level=level, t=epochs.times[t]))

all_scores = pd.DataFrame(all_scores,index=False)
all_evos = pd.DataFrame(all_evos,index=False)

all_scores.to_csv('./score.csv')
all_evos.to_csv('./evos.csv')


In [ ]:
all_scores = pd.DataFrame(all_scores)
all_evos = pd.DataFrame(all_evos)

In [ ]:
all_scores.query('level=="sentence"')

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(16, 10), dpi=80)

fig, axes = plt.subplots(3, 2)

for axes_, level in zip(axes, ('word', 'constituent', 'sentence')):
    for ax, start in zip( axes_, ('onset', 'offset')):  
        cond1 = all_scores.level==f'{level}'
        cond2 = all_scores.start==f'{start}'
        data = all_scores[ cond1 & cond2]
        print(data.shape)
        x = data['t']
        y = data['score']
        
        ax.plot(x,y)
        ax.set_title(f'{level} {start}')
        

In [ ]:
all_scores

###  #3
Now the same idea, but iterating on the targeted decoding: {word embedding, sentence embedding, etc..}

### #4 
Now baselined on offset, no matter whether it's on onset or offset window

### #5 
Submitit-compatible version

In [ ]:
!pip install submitit
import submitit

# Jitter test

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
import mne_bids
import mne
from pathlib import Path
subject = '5'
run_id = 1
path = get_path("LPP_read")
task = "read"
print(f"\n Epoching for run {run_id}, subject: {subject}\n")
bids_path = mne_bids.BIDSPath(
    subject=subject,
    session="01",
    task=task,
    datatype="meg",
    root=path,
    run=run_id,
)

raw = mne_bids.read_raw_bids(bids_path)
raw.del_proj()  # To fix proj issues
raw.pick_types(meg=True, stim=True)

# Generate event_file path
event_file = path / f"sub-{bids_path.subject}"
event_file = event_file / f"ses-{bids_path.session}"
event_file = event_file / "meg"
event_file = str(event_file / f"sub-{bids_path.subject}")
event_file += f"_ses-{bids_path.session}"
event_file += f"_task-{bids_path.task}"
event_file += f"_run-{bids_path.run}_events.tsv"
assert Path(event_file).exists()

events = mne.find_events(raw, stim_channel="STI101", shortest_event=1)

In [ ]:
events = mne.find_events(raw, stim_channel="STI101", shortest_event=1)
events_  = events[:,0] / raw.info["sfreq"]
diffs = np.diff(events_)
x,y = np.unique(diffs, return_counts=True)
plt.plot(x,y)
plt.xlim([0.2,0.3])
plt.show()


In [ ]:
raw.copy().pick_types(meg=False, stim=True).plot(start=50, duration=4)

In [ ]:
events = mne.find_events(raw, stim_channel="STI002", shortest_event=1)
events_  = events[:,0] / raw.info["sfreq"]
diffs = np.diff(events_)
x,y = np.unique(diffs, return_counts=True)
plt.plot(x,y)
plt.xlim([0.2,0.4])
plt.show()


In [ ]:
import matplotlib.pyplot as plt

meta = epochs.metadata

# Testing decoding more and more difficult

In [ ]:
from dataset import word_epochs_debug, get_path, get_subjects, sentence_epochs_debug
from utils import decod
from plot import plot_R
import mne

path = get_path("LPP_read")
subjects = get_subjects(path)
subjects = subjects[:2]

# WORDS
all_epochs = []
for sub in subjects:

    epochs = sentence_epochs_debug(sub, 3)
    all_epochs.append(epochs)

for epo in all_epochs:
    epo.info["dev_head_t"] = all_epochs[1].info["dev_head_t"]

epochs = mne.concatenate_epochs(all_epochs)


In [ ]:
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import RidgeCV
import numpy as np
from utils import correlate

def decod(X, y):
    assert len(X) == len(y)
    # define data
    model = make_pipeline(StandardScaler(), RidgeCV(alphas=np.logspace(-1, 6, 10)))
    cv = KFold(15, shuffle=True, random_state=0)

    # fit predict
    n, n_chans, n_times = X.shape
    if y.ndim == 1:
        y = np.asarray(y).reshape(y.shape[0], 1)
    R = np.zeros((n_times, y.shape[1]))

    for t in range(n_times):
        print(".", end="")
        rs = []
        # y_pred = cross_val_predict(model, X[:, :, t], y, cv=cv)
        for train, test in cv.split(X):
            model.fit(X[train, :, t], y[train])
            y_pred = model.predict(X[test, :, t])
            r = correlate(y[test], y_pred)
            rs.append(r)
        R[t] = np.mean(rs)
        # R[t] = correlate(y, y_pred)


    return R

# LASER

In [ ]:
from dataset import get_code_path

run_id = 1
CHAPTERS = {
1: "1-3",
2: "4-6",
3: "7-9",
4: "10-12",
5: "13-14",
6: "15-19",
7: "20-22",
8: "23-25",
9: "26-27",
}

meta = epochs.metadata

# # laser embeddings information
dim = 1024
embeds = np.fromfile(
    f"{get_code_path()}/data/laser_embeddings/emb_{CHAPTERS[int(run_id)]}.bin",
    dtype=np.float32,
    count=-1,
)
embeds.resize(embeds.shape[0] // dim, dim)
print(meta.shape[0])
embeds.shape


In [ ]:

epochs = epochs[]
X = epochs.get_data()
y = epochs.metadata.word.apply(len)
R_vec = decod(X, y)

## Word length

In [ ]:
# Remove the STIM information before decoding it (or else we'll get a 100% accuracy since the word length info is in the STIM channels)
epochs = epochs.pick_types(meg=True, stim=False, misc=False)
X = epochs.get_data()
y = epochs.metadata.word.apply(len)
R_vec = decod(X, y)

In [ ]:
fig = plot_R(R_vec.reshape(-1))

import matplotlib.pyplot as plt
plt.show()


## Word embeddings

In [ ]:
import spacy

nlp = spacy.load("fr_core_news_sm")
epochs = epochs.pick_types(meg=True, stim=False, misc=False)
X = epochs.get_data()
embeddings = epochs.metadata.word.apply(lambda word: nlp(word).vector).values
embeddings = np.array([emb for emb in embeddings])
R_vec = decod(X, embeddings)
R_vec = np.mean(R_vec, axis=1)

In [ ]:
fig = plot_R(R_vec.reshape(-1))

import matplotlib.pyplot as plt
plt.show()


## Laser embeddings with JR baseline fix

In [ ]:
sent_stop_data.shape
baseline_starts.shape

In [ ]:
sent_starts = epochs['word_id==0'].apply_baseline((-.300, 0.))
sent_starts.average().plot()

sent_stops = epochs['is_last_word']
bsl = (epochs.times>-.300 )*(epochs.times<=0)
baseline_starts = sent_starts.get_data()[:, :, bsl].mean(-2)

sent_stop_data = sent_stops.get_data()
n_sentences, n_channels, n_times = sent_stop_data.shape
sent_stop_data -= baseline_starts[:, :, None]


## Others

In [ ]:
epochs.metaepochs.metadata.word.apply(len)
decoding_criterion = "n_closing"
R_vec = decod(epochs, decoding_criterion)

fig = plot_R(R_vec)

import matplotlib.pyplot as plt
plt.show()

# Generate plots

In [ ]:
from dataset import get_path, get_subjects, word_epochs, sentence_epochs
import numpy as np
import matplotlib
import mne

matplotlib.use("Agg")

mne.set_log_level(False)


In [ ]:
report = mne.Report()
path = get_path("LPP_read")
subjects = get_subjects(path)
task = "read"
evos = []

# WORDS
subjects = subjects[2]
epochs = word_epochs(subjects)

evo = epochs.average(method="median")
evos.append(evo)
evo.plot(spatial_colors=True)
report.add_evokeds(evo, titles=f"Evoked for condition word  ")


# SENTENCES
epochs = sentence_epochs(subjects)

evo = epochs.average(method="median")
evos.append(evo)
evo.plot(spatial_colors=True)
report.add_evokeds(evo, titles=f"Evoked for condition sentence  ")


evokeds = dict(sentence=evos[1], word=evos[0])

fig = mne.viz.plot_compare_evokeds(evokeds, combine="mean")

report.add_figure(fig, title="Evoked response comparaison")


report.save(
    f"./figures/{task}_sentvsword_test.html",
    open_browser=False,
    overwrite=True,
)






In [ ]:
evo = epochs.average(method="median")
evo.plot(spatial_colors=True)

# Test new functions

In [ ]:
from dataset import word_epochs

sub = '3'

epochs = word_epochs(sub)


# Debug events

In [ ]:
import mne_bids
from pathlib import Path
import pandas as pd
import mne
from utils import match_list
from dataset import mne_events
from utils import add_syntax


CHAPTERS = {
    1: "1-3",
    2: "4-6",
    3: "7-9",
    4: "10-12",
    5: "13-14",
    6: "15-19",
    7: "20-22",
    8: "23-25",
    9: "26-27",
}

In [ ]:
path = get_path("LPP_read")
subjects = get_subjects(path)
run_id = 1
task = "read"
subject = '3'
baseline_min = -2.0
baseline_max = 0.5
epoch_on = 'sentence'
reference = "end"
print(f"\n Epoching for run {run_id}, subject: {subject}\n")
bids_path = mne_bids.BIDSPath(
    subject=subject,
    session="01",
    task=task,
    datatype="meg",
    root=path,
    run=run_id,
)

raw = mne_bids.read_raw_bids(bids_path)
raw.del_proj()  # To fix proj issues
raw.pick_types(meg=True, stim=True)
raw.load_data()
raw = raw.filter(0.5, 20)
# Generate event_file path
event_file = path / f"sub-{bids_path.subject}"
event_file = event_file / f"ses-{bids_path.session}"
event_file = event_file / "meg"
event_file = str(event_file / f"sub-{bids_path.subject}")
event_file += f"_ses-{bids_path.session}"
event_file += f"_task-{bids_path.task}"
event_file += f"_run-{bids_path.run}_events.tsv"
assert Path(event_file).exists()

# read events
meta = pd.read_csv(event_file, sep="\t")
events = mne.find_events(raw, stim_channel="STI101", shortest_event=1)

In [ ]:
meta_copy = meta.copy()

In [ ]:
meta = meta_copy

In [ ]:
path_syntax = get_code_path() / "data/syntax"

# Enriching the metadata with outside files:
meta = add_syntax(meta, path_syntax, int(run_id))

# Enriching the metadata with simple operations:

# end of sentence information
end_of_sentence = [
    True
    if str(meta.word.iloc[i]).__contains__(".")
    or str(meta.word.iloc[i]).__contains__("?")
    or str(meta.word.iloc[i]).__contains__("!")
    else False
    for i, _ in enumerate(meta.values[:-1])
]
end_of_sentence.append(True)
meta["sentence_end"] = end_of_sentence

# sentence start information
list_word_start = [True]
list_word_start_to_add = [
    True if meta.sentence_end.iloc[i - 1] else False
    for i in np.arange(1, meta.shape[0])
]
for boolean in list_word_start_to_add:
    list_word_start.append(boolean)
meta["sentence_start"] = list_word_start

# laser embeddings information
dim = 1024
embeds = np.fromfile(
    f"{get_code_path()}/data/laser_embeddings/emb_{CHAPTERS[int(run_id)]}.bin",
    dtype=np.float32,
    count=-1,
)
embeds.resize(embeds.shape[0] // dim, dim)
assert embeds.shape[0] == meta.shape[0]
meta["laser"] = [emb for emb in embeds]

# constituent end information
meta["constituent_end"] = [
        True if closing > 1 else False for i, closing in enumerate(meta.n_closing)]

# constituent start information
list_constituent_start = [True]
list_constituent_start_to_add = [
    True if meta.constituent_end.iloc[i - 1] else False
    for i in np.arange(1, meta.shape[0])
]
for boolean in list_constituent_start_to_add:
    list_constituent_start.append(boolean)
meta["constituent_start"] = list_constituent_start


In [ ]:
# word_length_meg = events[:, 2]
word_len_meta = meta.word.apply(len)
i, j = match_list(word_len_meta, word_length_meg)
events = events[j]
assert len(i) / meta.shape[0] > 0.8
meta = meta.iloc[i].reset_index()

In [ ]:
meta["start"] = events[:, 0] / raw.info["sfreq"]
meta["condition"] = "sentence"
meta = meta.sort_values("start").reset_index(drop=True)
meta["word_start"] = meta["start"]
meta["word_end"] = meta["word_start"] + meta["duration"]

epochs = mne.Epochs(
    raw, **mne_events(meta, raw), decim=20, tmin=baseline_min, tmax=baseline_max
)

In [ ]:
meta

# Plotting decoding info

In [ ]:
from plot import plot_subject

from dataset import get_path, get_subjects, get_code_path
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
task = "read"
sub = 4
epoch_on = 'sentence'
reference = "end"
min = -4.0
max = 0.5
decoding_criterion = 'laser'
path = get_code_path()
# Format the file path

# Open the pandas DataFrame containing the decoding values
R = np.load(
    (path) / f"decoding/results/{task}/decoding_{decoding_criterion}_{epoch_on}_{reference}_{sub}.npy"
)

In [ ]:
R

In [ ]:
times = np.linspace(min, max, R.shape[0])  # To do better at generalizing
fig, ax = plt.subplots(1, figsize=[6, 6])
dec = plt.fill_between(times, R)

In [ ]:
task = "read"
sub = 3
epoch_on = 'sentence'
reference = "end"
min = -4.0
max = 0.5
decoding_criterion = 'laser'
plot = plot_subject(sub, decoding_criterion, task, reference, epoch_on, min, max)

# Debugging ERP plots

In [ ]:
path = get_path("LPP_read")
subjects = get_subjects(path)
RUN = 1
baseline_min = -1.0
baseline_max = 1.0
task = "read"

subjects = subjects[10]
epochs2 = epoch_subjects(
    subjects, RUN, task, path, baseline_max=baseline_max, baseline_min=baseline_min
)

epochs2

In [ ]:
epochs2.metadata

In [ ]:
# Homemade imports
from dataset import get_path, get_subjects, epoch_subjects, epochs_slice
from plot import plot_subject

# General imports
import numpy as np
import mne
import hydra
from omegaconf import DictConfig
import matplotlib

matplotlib.use("Agg")

mne.set_log_level(False)

# Later: integrate Hydra here as well. For now, just simple plotting of ERPS

report = mne.Report()
path = get_path("LPP_read")
subjects = get_subjects(path)
RUN = 1
baseline_min = -1.0
baseline_max = 1.0
task = "read"
print("\nSubjects for which the plotting will be done: \n")
print(subjects)

# DEBUG
subjects = subjects[4]
epochs_ = epoch_subjects(
    subjects, RUN, task, path, baseline_max=baseline_max, baseline_min=baseline_min
)






# Build a 3x2 plot, with for each condition (sentence, word, constituent), and for (start, end),
# the ERP associated
cond = ["sentence", "word", "constituent"]
cases = {"start", "end"}

# Plotting and adding to the report, the averaged ERPs of:
# words, sentences and constituents, centered at the beginning and end of each


# Need to map for:
# - end of word,
# - beginning of sentence (epochs[i+1] !danger limits)
# - beginning of constituent (epochs[i+1] !same danger)

evos = []
for condi in cond:
    for case in cases:
        # Slice the epochs based on the epoch_criterion:
        column_to_slice_on = f"{condi}_{case}"
        if condi == "sentence" or condi == "word":  # eg: {sentence}_{end} or {word}_{start}
            epochs = epochs_slice(epochs_, column_to_slice_on)
        elif condi == "constituent":
            epochs = epochs_slice(epochs_, column_to_slice_on, value=2, equal='sup')
        evo = epochs.average(method="median")
        evos.append(evo)
        evo.plot(spatial_colors=True)
        report.add_evokeds(evo, titles=f"Evoked for condition {column_to_slice_on}  ")

evokeds = dict(sentence=evos[0], word=evos[2], constituent=evos[4])

fig = mne.viz.plot_compare_evokeds(evokeds, combine="mean")

report.add_figure(fig, title="Evoked response comparaison")


report.save(
    f"./figures/{task}_ERP_all_cond.html",
    open_browser=False,
    overwrite=True,
)

In [ ]:
epochs_

In [ ]:
ep = epochs_slice(epochs_, 'sentence_start')

In [ ]:
ep

In [ ]:
ep

In [ ]:
from plot import plot_subject

from dataset import get_path, get_subjects

In [ ]:
path = get_path("LPP_read")
subjects = get_subjects(path)
run = 1
task = "read"
subject = '17'
baseline_min = -2.0
baseline_max = 0.5
epoch_on = 'sentence'
reference = "end"

In [ ]:
from dataset import epoch_data
epo = epoch_data(
    subject,
    run,
    task,
    path,
    baseline_min,
    baseline_max,
)

In [ ]:
from dataset import epochs_slice
epos = epochs_slice(epo, 'sentence_end')

In [ ]:
epos

In [ ]:
evo = epos.average(method="median")
evo.plot()

# GFP for sentence - epoching on sentence end and go from there

In [ ]:
# Homemade imports
from dataset import get_path, get_subjects, epoch_runs
from plot import plot_subject

# General imports
import numpy as np
import mne
import hydra
from omegaconf import DictConfig
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler
from wordfreq import zipf_frequency
from Levenshtein import editops

# Tools
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib
from utils import match_list, add_syntax
import spacy

In [ ]:
path = get_path("LPP_read")
subjects = get_subjects(path)
RUN = 2
task = "read"
subject = subjects[1]
baseline_min = -4.0
baseline_max = 0.5
epoch_on = 'sentence'
reference = "end"

In [ ]:
epochs = epoch_runs(
            subject,
            RUN,
            task,
            path,
            baseline_min,
            baseline_max,
            epoch_on=epoch_on,
            reference=reference,
        )

In [ ]:
run = 1
from dataset import epoch_data
epo = epoch_data(
    subject,
    run,
    task,
    path,
    baseline_min=-0.2,
    baseline_max=0.8,
    filter=True,
    epoch_on="word",
    reference="end",
)

In [ ]:
meta = epo.metadata

In [ ]:

meta[50:100]

In [ ]:
from dataset import get_code_path, get_path, mne_events

run_id = RUN
epoch_on = 'word'
reference = "start"

CHAPTERS = {
    1: "1-3",
    2: "4-6",
    3: "7-9",
    4: "10-12",
    5: "13-14",
    6: "15-19",
    7: "20-22",
    8: "23-25",
    9: "26-27",
}


bids_path = mne_bids.BIDSPath(
        subject=subject,
        session="01",
        task=task,
        datatype="meg",
        root=path,
        run=RUN,
    )

raw = mne_bids.read_raw_bids(bids_path)
raw.del_proj()  # To fix proj issues
raw.pick_types(meg=True, stim=True)
raw.load_data()
raw = raw.filter(0.5, 20)
# Generate event_file path
event_file = path / f"sub-{bids_path.subject}"
event_file = event_file / f"ses-{bids_path.session}"
event_file = event_file / "meg"
event_file = str(event_file / f"sub-{bids_path.subject}")
event_file += f"_ses-{bids_path.session}"
event_file += f"_task-{bids_path.task}"
event_file += f"_run-{bids_path.run}_events.tsv"
assert Path(event_file).exists()

# read events
meta = pd.read_csv(event_file, sep="\t")
events = mne.find_events(raw, stim_channel="STI101", shortest_event=1)
if (
    bids_path.task == "read" and bids_path.subject == "2"
):  # A trigger value bug for this subject
    word_length_meg = (
        events[:, 2] - 2048
    )  # Remove first event: chapter start and remove offset
else:
    word_length_meg = events[:, 2]
# Here, the trigger value encoded the word length
# which helps us realign triggers
# From the event file / from the MEG events
word_len_meta = meta.word.apply(len)
i, j = match_list(word_len_meta, word_length_meg)
events = events[j]
meta = meta.iloc[i].reset_index()
print(meta.shape)
# The start parameter will help us
# keep the link between raw events and metadata
meta["start"] = events[:, 0] / raw.info["sfreq"]
meta["condition"] = "sentence"
meta = meta.sort_values("start").reset_index(drop=True)

# Raw LPP textual data
path_txt = get_code_path() / "data/txt_raw"
# LPP Syntax data
path_syntax = get_code_path() / "data/syntax"

# Enriching the metadata with outside files:
meta = add_syntax(meta, path_syntax, int(run_id))
print(meta.shape)
# Add the information on the sentence ending:
# Only works for reading: TO FIX for listening... to see with Christophe
# Also: only works for v2 (subject 1 (me) doesn't work )

# Test 1
# end_of_sentence = [
#     True if meta.onset.iloc[i + 1] - meta.onset.iloc[i] > 0.7 else False
#     for i, _ in enumerate(meta.values[:-1])
# ]
# end_of_sentence.append(True)

# Test 2
end_of_sentence = [
    True
    if str(meta.word.iloc[i]).__contains__(".")
    or str(meta.word.iloc[i]).__contains__("?")
    or str(meta.word.iloc[i]).__contains__("!")
    else False
    for i, _ in enumerate(meta.values[:-1])
]
end_of_sentence.append(True)
meta["sentence_end"] = end_of_sentence

# We are considering different cases:
# Are we epoching on words, sentences, or constituents?
# Different epoching for different analysis
if epoch_on == "word" and reference == "start":
    # Default case, so nothing to change
    # Could be removed but kept for easy of reading
    happy = True
# Word end
if epoch_on == "word" and reference == "end":
    # Little hack: not really pretty but does the job
    # As epoching again uses the start column, we rename it like that
    # But it should be meta["end"] instead...
    meta["start"] = [row["start"] + row["duration"] for i, row in meta.iterrows()]

# Sentence end
elif epoch_on == "sentence" and reference == "end":
    # Add a LASER embeddings column for decoding
    dim = 1024
    embeds = np.fromfile(
        f"{get_code_path()}/data/laser_embeddings/emb_{CHAPTERS[int(run_id)]}.bin",
        dtype=np.float32,
        count=-1,
    )
    embeds.resize(embeds.shape[0] // dim, dim)
    column = "sentence_end"
    value = True
    meta = meta[meta[column] == value]
    # TODO: create a match list between the embeds sentence and the
    print(embeds.shape[0], meta.shape[0])
    assert embeds.shape[0] == meta.shape[0]
    meta["laser"] = [emb for emb in embeds]
    print("Added embeddings")
# Sentence start
elif epoch_on == "sentence" and reference == "start":
    # Create a sentence-start column:
    # list_word_start = [
    #     True
    #     for i, is_last_word in enumerate(meta.is_last_word[:-1])
    #     if meta.is_last_word[i + 1]
    # ]
    list_word_start = [True]
    list_word_start_to_add = [
        True if meta.sentence_end[i - 1] else False
        for i, _ in enumerate(meta.sentence_end[1:])
    ]
    for boolean in list_word_start_to_add:
        list_word_start.append(boolean)
    meta["sentence_start"] = list_word_start
    column = "sentence_start"
    value = True
    meta = meta[meta[column] == value]
# Constituent start
elif epoch_on == "constituent" and reference == "start":
    # Create a constituent-start column:
    meta["constituent_start"] = [
        True for i, _ in enumerate(meta.is_last_word[1:]) if meta.n_closing > 1
    ]
    column = "constituent_start"
    value = True
    meta = meta[meta[column] == value]
# Constituent end
elif epoch_on == "constituent" and reference == "end":
    # Create a constituent-start column:
    meta["constituent_start"] = [
        True for i, _ in enumerate(meta.is_last_word[1:]) if meta.n_closing > 1
    ]
    column = "constituent_start"
    value = True
    meta = meta[meta[column] == value]
epochs = mne.Epochs(
    raw, **mne_events(meta, raw), decim=20, tmin=baseline_min, tmax=baseline_max
)
# epochs = epochs['kind=="word"']
# epochs.metadata["closing"] = epochs.metadata.closing_.fillna(0)
epochs.load_data()
epochs = epochs.pick_types(meg=True, stim=False, misc=False)
data = epochs.get_data()

# Scaling the data
n_words, n_chans, n_times = data.shape
vec = data.transpose(0, 2, 1).reshape(-1, n_chans)
scaler = RobustScaler()
idx = np.arange(len(vec))
np.random.shuffle(idx)
vec = scaler.fit(vec[idx[:20_000]]).transform(vec)
# To try: sigmas = 7 or 15
sigma = 7
vec = np.clip(vec, -sigma, sigma)
epochs._data[:, :, :] = (
    scaler.inverse_transform(vec)
    .reshape(n_words, n_times, n_chans)
    .transpose(0, 2, 1)
)

In [ ]:
meta[50:100]

In [ ]:
j.shape

In [ ]:
events.shape

In [ ]:
events_2 = mne.find_events(raw, stim_channel="STI101", shortest_event=1)
events_2.shape

In [ ]:
b.shape

In [ ]:
epochs.metadata

In [ ]:
from utils import decod
R_vec = decod(epochs, decoding_criterion)

In [ ]:
import matplotlib.pyplot as plt
plt.fill_between(epochs.times, R_vec)

In [ ]:
epochs.metadata

In [ ]:
np.diff(epochs.metadata.onset)

In [ ]:
epochs2 = epoch_runs(subject, RUN, task, path, baseline_min,baseline_max)

x 
x 
x 
x 
x 
x 
x 
x 
x 

In [ ]:
arr = np.round(np.diff(epochs2.metadata.onset),3)
unique, counts = np.unique(arr, return_counts=True)

# Print unique values and their counts
for val, count in zip(unique, counts):
    print(f"{val} occurs {count} times.")


In [ ]:
sum(np.diff(epochs2.metadata.onset)>0.7)

In [ ]:
evo = epochs.average(method="median")
evo.plot(gfp='only')

In [ ]:
epochs.events

In [ ]:
def epochs_(epochs, column, value):
    meta  = epochs.metadata
    subset = meta[meta[column]==value].level_0
    return epochs[subset]


In [ ]:
epochs.metadata['n_closing'][:50]

In [ ]:
epochs_(epochs,'is_last_word',True)

In [ ]:
import matplotlib.pyplot as plt
# Build a 3x2 plot, with for each condition (sentence, word, constituent), and for (start, end),
# the ERP associated
cond = {'sentence': {'column':'is_last_word','target':True},
        'word': {'column':'kind','target':'word'},
        'constituent': {'column':'n_closing','target':2}}

cases = {'start', 'end'}

i = 1
for condi in cond:
    for case in cases:
        ep = epochs_(epochs, cond[condi]['column'], cond[condi]['target'])
        ax = fig.add_subplot(3, 2, i)
        #ep.average().plot(gfp='only')
        evo = ep.average(method="median")
        evo.plot(spatial_colors=True)
        i = i + 1
        ax.set_title(f'Plot {cond}')


plt.show()


In [ ]:
epochs_4 = epochs_(epochs, 'n_closing', 4)

In [ ]:
epochs_4.average().plot(gfp='only')

In [ ]:
evo = epochs.average(method="median")
evo.plot(spatial_colors=True)

In [ ]:
metadata_subset

In [ ]:
from mne import Epochs

class CustomEpochs(Epochs):

    def __getitem__(self, key):
        # Parse the key into metadata field name and value
        field, value = key.split('==')
        field = field.strip()
        value = value.strip()

        # Get the indices of the epochs that match the metadata query
        indices = [i for i, metadata in enumerate(self.metadata[field]) if metadata == value]

        # Return a new Epochs object containing only the matching epochs
        return self.__class__(self._data[indices], self.events[indices], self.event_id,
                              tmin=self.tmin, tmax=self.tmax, baseline=self.baseline,
                              metadata=self.metadata.iloc[indices], info=self.info)



In [ ]:
custom_epochs = CustomEpochs(epochs, epochs.events, "1", -0.2, 0.8, epochs.baseline, epochs.metadata)

# Get all epochs where the 'kind' metadata field is 'word':
word_epochs = custom_epochs['kind==word']

In [ ]:
custom_epochs['kind=="word"']

In [ ]:
# epoch
def mne_events(meta):
    events = np.ones((len(meta), 3), dtype=int)
    events[:, 0] = meta.start*raw.info['sfreq']
    return dict(events=events, metadata=meta.reset_index())

epochs = mne.Epochs(raw, **mne_events(meta), decim=20, tmin=-.2, tmax=1.5, preload=True)
epochs = epochs['kind=="word"']

In [ ]:
from dataset import get_path, get_subjects, epoch_data, epoch_runs
from utils import (
    decod,
    correlate,
    match_list,
    create_target,
    analysis,
    save_decoding_results,
)
from plot import plot_subject
import mne_bids
from pathlib import Path
import pandas as pd
import numpy as np
import mne
import spacy
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import RidgeCV
from wordfreq import zipf_frequency
from Levenshtein import editops
import matplotlib.pyplot as plt

In [ ]:
path = get_path("LPP_read")
subjects = get_subjects(path)
RUN = 9
task = "read"
subject = subjects[4]

In [ ]:
epochs = epoch_runs(subject, RUN, task, path)

In [ ]:
(epochs.metadata).dtype

In [ ]:
epochs.load_data()
epochs = epochs['kind=="word"']
epochs["content_word == False"]